In [ ]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import os

import math

In [ ]:
def calculate_angle(center_landmark, start_landmark, end_landmark):

    center = np.array(center_landmark)
    start = np.array(start_landmark)
    end = np.array(end_landmark)
    

    vector1 = start - center
    vector2 = end - center
    

    angle_radians = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
    

    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees


def draw_selected_landmarks_on_image(rgb_image, landmarks, selected_landmarks, connections,labels):

    annotated_image = np.copy(rgb_image)


    for index in selected_landmarks:
        landmark = landmarks.landmark[index]
        x, y, z = landmark.x, landmark.y, landmark.z

        x_pixel, y_pixel = int(x * annotated_image.shape[1]), int(y * annotated_image.shape[0])
        if(( (index in [28,24,26]))or( (index in [12,24,28]))):
            cv2.circle(annotated_image, (x_pixel, y_pixel), radius=5, color=(0, 255, 0), thickness=-1)
        else:
            cv2.circle(annotated_image, (x_pixel, y_pixel), radius=5, color=(0, 0, 255), thickness=-1)

    for connection in connections:
        start_index, end_index = connection
        start_landmark = landmarks.landmark[start_index]
        end_landmark = landmarks.landmark[end_index]
        start_x, start_y = int(start_landmark.x * annotated_image.shape[1]), int(start_landmark.y * annotated_image.shape[0])
        end_x, end_y = int(end_landmark.x * annotated_image.shape[1]), int(end_landmark.y * annotated_image.shape[0])
        # cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        # if(label):
        #     cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        # else:
        #     cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 0, 255), 2)
        if((labels[0] and ((start_index==24 and end_index==26) or (start_index==26 and end_index==24) or (start_index==26 and end_index==28) or (start_index==28 and end_index==26) )) or (labels[1] and ((start_index==12 and end_index==24) or (start_index==24 and end_index==12) or (start_index==24 and end_index==28) or (start_index==28 and end_index==24) )) ):
            cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        else:
            cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 0, 255), 2)
            

    return annotated_image

In [ ]:
mp_pose = mp.solutions.pose
qiyam_data = []
hip_threshold = 160
knee_threshold = 160
selected_landmarks_indices = [16,14,12,24,26,28,32]
selected_landmarks_connections = [(16,14),(14,12),(24,26),(26,28),(12,24),(24,28),(28,32)]

video_directory = 'videos'

In [ ]:
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1) as pose:
    for video_file in os.listdir(video_directory):
        if video_file.endswith(".mp4"):
            video_path = os.path.join(video_directory, video_file)
            cap = cv2.VideoCapture(video_path)

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                     landmark_12 = (results.pose_landmarks.landmark[selected_landmarks_indices[2]].x,
                                    results.pose_landmarks.landmark[selected_landmarks_indices[2]].y)
                     landmark_24 = (results.pose_landmarks.landmark[selected_landmarks_indices[3]].x,
                                    results.pose_landmarks.landmark[selected_landmarks_indices[3]].y)
                     landmark_26 = (results.pose_landmarks.landmark[selected_landmarks_indices[4]].x,
                                    results.pose_landmarks.landmark[selected_landmarks_indices[4]].y)
                     
                     hip_angle = calculate_angle(landmark_24, landmark_12, landmark_26)
                     annotated_frame= draw_selected_landmarks_on_image(frame, results.pose_landmarks, selected_landmarks_indices, selected_landmarks_connections)

                    
                    # shoulder_hip_label = 'correct' if shoulder_hip_angle > hip_threshold else 'incorrect'
                    # hip_knee_label = 'correct' if hip_knee_angle > knee_threshold else 'incorrect'

                    # qiyam_data.append([shoulder_hip_angle, hip_knee_angle, shoulder_hip_label, hip_knee_label])

                if cv2.waitKey(1) & 0xFF == 27: 
                    break

    
            cap.release()


# qiyam_df = pd.DataFrame(qiyam_data, columns=['Shoulder_Hip_Angle', 'Hip_Knee_Angle', 'Shoulder_Hip_Label', 'Hip_Knee_Label'])
# qiyam_df.to_csv('qiyam_dataset.csv', index=False)
cv2.destroyAllWindows()